## Co-moving stars in Kepler

(done in a very hacky way) (but this is probably ok because we're assuming all the stars are far away)

In [ ]:
import numpy as np
from astropy.table import Table, unique
from astropy import units as u
import astropy.coordinates as coord
from astropy.time import Time
from astropy.io import fits
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd

Load up the table of all Gaia DR2 sources within the Kepler field:

In [ ]:
gaia_table_file = '../data/gaia-kepler-dustin.fits'
hdul = fits.open(gaia_table_file)
gaia_src_tbl = Table(hdul[1].data)

In [ ]:
min_columns = ['source_id', 'ra', 'dec', 'parallax', 'pmra', 'pmdec', 'parallax_error',
                'pmra_error', 'pmdec_error', 'parallax_pmra_corr', 'parallax_pmdec_corr',
                'pmra_pmdec_corr']
min_table = gaia_src_tbl[min_columns]

In [4]:
min_table = min_table.to_pandas()
#full_table = gaia_src_tbl.to_pandas()

Now load up the Gaia-Kepler cross-match so we can add Kepler IDs to the best-match Gaia sources:

In [ ]:
kepler_table_file = '../data/kepler_dr2_1arcsec.fits'
hdul = fits.open(kepler_table_file)
kepler_tbl = Table(hdul[1].data)
gaia_kepler_matches = kepler_tbl['kepid', 'kepler_gaia_ang_dist', 'source_id', 'nconfp', 'nkoi', 'planet?']
gaia_kepler_matches = gaia_kepler_matches.to_pandas()
print(len(gaia_kepler_matches))

Trim off the less-good matches so that there's one unique Gaia source per Kepler target:

In [ ]:
gaia_kepler_matches.sort_values(['kepid', 'kepler_gaia_ang_dist'], inplace=True)
gaia_kepler_matches.drop_duplicates('kepid', inplace=True)
print(len(gaia_kepler_matches))

And join the tables:

In [ ]:
full_table = full_table.merge(gaia_kepler_matches, on='source_id', how='left')
len(full_table)

In [ ]:
test_id = 2105885485289168768
print(full_table[full_table['source_id'] == test_id])

Now load up Dustin's pairs:

In [8]:
pairs_file = '../data/matched-pairs-dustin.fits'
hdul = fits.open(pairs_file)
pairs = hdul[0].data

In [9]:
pairs[:10]

array([[6750, 6753,    1, ...,   -1,   -1,   -1],
       [6750, 6753,    5, ...,   -1,   -1,   -1],
       [   3,    5, 1446, ...,   -1,   -1,   -1],
       ...,
       [   3,    2, 1446, ...,   -1,   -1,   -1],
       [6755,    2, 1446, ...,   -1,   -1,   -1],
       [  10,   13,  333, ...,   -1,   -1,   -1]], dtype=int32)

In [11]:
pairs = pd.DataFrame(data=pairs)
pairs.iloc[:10]

,0,1,2,3,4,5,6,7,8,9,...,159,160,161,162,163,164,165,166,167,168
0,6750,6753,1,6754,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,6750,6753,5,0,6754,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,3,5,1446,4,8,7,6,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,5,2,4,7,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,3,5,2,1446,7,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
5,3,1,2,4,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
6,6755,2,1446,8,6767,6766,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
7,3,2,1446,4,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
8,6755,2,1446,6,6767,6766,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
9,10,13,333,342,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


Define some useful functions:

In [12]:
def make_x(star):
    """
    returns a vector of x = [parallax, pmra, pmdec]
    """
    names = ['parallax', 'pmra', 'pmdec']
    return star.loc[names].values.astype('f')

def make_xerr(star):
    """
    returns a vector of xerr = [parallax_error, pmra_error, pmdec_error]
    """
    err_names = ['parallax_error', 'pmra_error', 'pmdec_error']
    return star.loc[err_names].values.astype('f')    

In [13]:
def ppm_check(star1, star2, sigma=5.):
    """
    Returns True if the differences between parallax, pmra, and pmdec are all below 
    the sigma threshold.
    """
    x1 = make_x(star1)
    x2 = make_x(star2)
    if np.any(np.isnan([x1,x2])):
        return False
    xerr1 = make_xerr(star1)
    xerr2 = make_xerr(star2)
    if np.any(np.isnan([xerr1, xerr2])):
        return False
    if np.any(np.abs(x1 - x2)/np.sqrt(xerr1**2 + xerr2**2) >= sigma):
        return False
    return True

In [14]:
def make_cov(star):
    """
    returns covariance matrix C corresponding to x
    """
    names = ['parallax', 'pmra', 'pmdec']
    C = np.diag(make_xerr(star)**2)
    for i, name1 in enumerate(names):
        for j, name2 in enumerate(names):
            if j >= i:
                continue
            corr = star.loc["{0}_{1}_corr".format(name2, name1)]
            C[i, j] = corr * np.sqrt(C[i, i] * C[j, j])
            C[j, i] = C[i, j]
    return C

def chisq(star1, star2):
    """
    calculates chisquared for two stars based on their parallax and 2D proper motions
    """
    deltax = make_x(star1) - make_x(star2)
    cplusc = make_cov(star1) + make_cov(star2)
    return np.dot(deltax, np.linalg.solve(cplusc, deltax))

In [15]:
def check_with_primary(m, primary):
    if ppm_check(primary, m):
        return chisq(primary, m)
    else:
        return -1

Now calculate a goodness-of-fit metric for each pair, skipping over the ones that don't make an initial cut:

In [ ]:
%%time
chisqs = np.zeros_like(pairs) - 1.

for i,row in tqdm(enumerate(pairs[:100])):
    primary = min_table.iloc[i]
    row_mask = (row > -1) & (row > i) # indices in row for matches to compute
    matches = min_table.iloc[row[row_mask]] # ignore non-matches and duplicates
    if np.sum(row_mask) > 0:
        row_of_chisqs = matches.apply(check_with_primary, args=(primary,), axis=1)
        chisqs[i,row_mask] = row_of_chisqs.values


Save the outputs and take a look at their distribution:

In [ ]:
hdu = fits.PrimaryHDU(chisqs)
hdulist = fits.HDUList([hdu])
hdulist.writeto('../data/chisqs_matched-pairs.fits')
hdulist.close()

In [ ]:
# optional - load up already-saved outputs
#hdul = fits.open('../data/chisqs_matched-pairs.fits')
#chisqs = hdul[0].data

In [ ]:
plt.hist(chisqs[(chisqs > 0.) & (chisqs < 10.)], bins=100)
plt.xlabel('$\chi^2$', fontsize=16)
plt.ylabel('# Pairs', fontsize=16)
plt.yscale('log')
plt.savefig('chisq_keplerpairs.png')

OK, now let's select the best-fit pairs and save their indicies for easy access:

In [ ]:
matches_mask = (chisqs > 0) & (chisqs < 2)
np.sum(matches_mask)

In [ ]:
len_inds, len_matches = np.shape(pairs)
pairs_inds = np.array([np.arange(len_inds),]*len_matches).transpose()
pairs_ind1s = pairs_inds[matches_mask]
pairs_ind2s = pairs[matches_mask]

In [ ]:
def read_match_attr(ind1, ind2, attr):
    return table.iloc[ind1][attr], table.iloc[ind2][attr]

In [ ]:
print("source_ids of a pair:")
print(read_match_attr(pairs_ind1s[0], pairs_ind2s[0], 'source_id'))

Sanity check: plot the parallax and proper motions of an identified match

In [ ]:
from plot_tools import error_ellipse
fs = 12

In [ ]:
def plot_xs(i, sigma=1):
    star1 = table.iloc[pairs_ind1s[i]]
    star2 = table.iloc[pairs_ind2s[i]]
    x1 = make_x(star1)
    cov1 = make_cov(star1)
    x2 = make_x(star2)
    cov2 = make_cov(star2)
    fig = plt.figure(figsize=(12,4))
    ax1 = fig.add_subplot(131)
    error_ellipse(ax1, x1[0], x1[1], cov1[:2,:2], ec='red', sigma=sigma)
    error_ellipse(ax1, x2[0], x2[1], cov2[:2,:2], ec='blue', sigma=sigma)
    ax1.set_xlim([min([x1[0], x2[0]]) - 5., max([x1[0], x2[0]]) + 5.])
    ax1.set_ylim([min([x1[1], x2[1]]) - 5., max([x1[1], x2[1]]) + 5.])
    ax1.set_xlabel('Parallax (mas)', fontsize=fs)
    ax1.set_ylabel('PM RA (mas yr$^{-1}$)', fontsize=fs)

    ax2 = fig.add_subplot(133)
    error_ellipse(ax2, x1[1], x1[2], cov1[1:,1:], ec='red', sigma=sigma)
    error_ellipse(ax2, x2[1], x2[2], cov2[1:,1:], ec='blue', sigma=sigma)
    ax2.set_xlim([min([x1[1], x2[1]]) - 5., max([x1[1], x2[1]]) + 5.])
    ax2.set_ylim([min([x1[2], x2[2]]) - 5., max([x1[2], x2[2]]) + 5.])
    ax2.set_xlabel('PM RA (mas yr$^{-1}$)', fontsize=fs)
    ax2.set_ylabel('PM Dec (mas yr$^{-1}$)', fontsize=fs)
    
    ax3 = fig.add_subplot(132)
    c1 = np.delete(np.delete(cov1, 1, axis=0), 1, axis=1)
    c2 = np.delete(np.delete(cov2, 1, axis=0), 1, axis=1)
    error_ellipse(ax3, x1[0], x1[2], c1, ec='red', sigma=sigma)
    error_ellipse(ax3, x2[0], x2[2], c2, ec='blue', sigma=sigma)
    ax3.set_xlim([min([x1[0], x2[0]]) - 5., max([x1[0], x2[0]]) + 5.])
    ax3.set_ylim([min([x1[2], x2[2]]) - 5., max([x1[2], x2[2]]) + 5.])
    ax3.set_xlabel('Parallax (mas)', fontsize=fs)
    ax3.set_ylabel('PM Dec (mas yr$^{-1}$)', fontsize=fs)
    
    fig.subplots_adjust(wspace = 0.5)
    fig.text(0.5, 0.95, 'match #{0}'.format(i), horizontalalignment='center', 
             transform=ax3.transAxes, fontsize=fs+2)

In [ ]:
i = np.random.randint(0, len(pairs_ind1s))
print("match {0}: source_ids {1}".format(i, 
            read_match_attr(pairs_ind1s[i], pairs_ind2s[i], 'source_id')))
plot_xs(i, sigma=3)

In [ ]:
pd.options.display.max_columns = None
src1, src2 = read_match_attr(pairs_ind1s[i], pairs_ind2s[i], 'source_id')
table[table['source_id'].isin([src1, src2])]

In [ ]:
print("saved chisquared = {0:.5f}".format(chisqs[pairs_ind1s[i]][np.where(pairs[pairs_ind1s[i]] 
                                                                    == pairs_ind2s[i])[0][0]]))


In [ ]:
star1 = table.iloc[pairs_ind1s[i]]
star2 = table.iloc[pairs_ind2s[i]]
chisq(star1, star2)

Let's look at the relative luminosities of each match:

In [ ]:
(gmag1, gmag2) = read_match_attr(pairs_ind1s, pairs_ind2s, 'phot_g_mean_mag')
(plx1, plx2) = read_match_attr(pairs_ind1s, pairs_ind2s, 'parallax')
dist1 = 1.e3/plx1
absg1 = gmag1 - 5.*(np.log10(dist1) - 1.)
dist2 = 1.e3/plx2
absg2 = gmag2 - 5.*(np.log10(dist2) - 1.)

Select only the ones with measured G:

In [ ]:
mask = np.all(np.vstack([np.isfinite(absg1), np.isfinite(absg2)]), axis=0)
good_pairs_2d = np.vstack([absg1[mask], absg2[mask]])
#good_pairs_2d = np.sort(good_pairs_2d, axis=0) # we could sort by brightness here
absg1, absg2 = good_pairs_2d[0], good_pairs_2d[1]

In [ ]:
absg = np.append(absg1, absg2)
hist = plt.hist(absg, bins=500)
plt.xlim([-5,15])
plt.xlabel('G')
plt.ylabel('# of stars')
plt.savefig('absmag_hist.png')

In [ ]:
from matplotlib.colors import LogNorm
plt.hist2d(absg1, absg2, bins=(1000,1000), norm=LogNorm())
cbar = plt.colorbar()
cbar.ax.set_ylabel('# of stars', rotation=270)
plt.xlabel('G$_{1}$')
plt.ylabel('G$_{2}$')
plt.xlim([-5, 15])
plt.ylim([-5, 15])
plt.savefig('absmag_pairs.pdf')

Now let's see how many of the matches are in the Kepler catalog, and whether any of them have planets!

In [ ]:
ind1_is_kic = np.isfinite(table.iloc[pairs_ind1s]['kepid'])
ind2_is_kic = np.isfinite(table.iloc[pairs_ind2s]['kepid'])

In [ ]:
one_is_kic = np.any(np.vstack([ind1_is_kic, ind2_is_kic]), axis=0)
both_are_kic = np.all(np.vstack([ind1_is_kic, ind2_is_kic]), axis=0)

In [ ]:
np.sum(both_are_kic)

In [ ]:
for i1, i2 in zip(pairs_ind1s[both_are_kic], pairs_ind2s[both_are_kic]):
    print(read_match_attr(i1,i2,'planet?'))